# **PyTorch Osnove 4: Primer Rešavanja Konkretnog Problema**

**Kurs:** KSMF1  
**Trajanje:** ~45 min  
**Preduslovi:** Pytorch Osnove 1, 2 i 3  

---

## Ciljevi Sekcije

Do kraja ove sekcije, naučićete da:

Here's the Serbian translation:

- Primeniti PyTorch na realističan problem klasifikacije u fizici
- Profesionalno koristiti PyTorch apstrakcije podataka (Dataset, DataLoader)
- Graditi i trenirati neuronske mreže za multiklasnu klasifikaciju
- Proceniti performanse klasifikacije koristeći standardne metrike
- Implementirati kompletan ML radni tok od podataka do implementacije

---

# **Deo 7: Praktičan Primer - Problem Identifikacije Čestica**

## 7.1 Uvod

Dobrodošli u **CMS izazov klasifikacije čestica**! 🔬

Zamislite da radite na eksperimentu iz fizike čestica (CERN-ov Veliki hadronski sudarač). Kada visokoenergetske čestice prođu kroz detektor, možete da merite različita svojstva.

Vaš zadatak: Napraviti ML klasifikator koji će **klasifikovati koji tip čestice je prošao kroz detektor** na osnovu ovih merenja.

**Izazov:**
- **Čestice:** Elektroni, Mioni, Pioni (3 klase)
- **Merenja:** Depozit energije, zakrivljenost traga, vreme leta, obrazac pogodaka (4 karakteristike)
- **Cilj:** Izgraditi klasifikator sa >90% tačnošću

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Postavi random seed zbog reproducibilnosti
torch.manual_seed(42)
np.random.seed(42)

print("🔬 CMS Particle Classification Challenge")
print("Goal: Classify particles from detector readings")

---

## 7.2 Prikupljanje podataka

Svaki Machine Learning workflow počinje sa prikupljanjem, analizom, i pripremom podataka!

### Generisanje Realističnih Podataka Čestica

Pošto nemamo pristup pravim eksperimentalnim podacima (i oni su mnogo složeniji od ovog primera), hajde da **simuliramo podatke iz detektora čestica**, sa pojednostavljenim korelacijama i obrascima šuma, čisto da imamo sa čime da radimo.

Generisaćemo 2000 uzoraka realističnih podataka detektora čestica sa pojednostavljenim korelacijama zasnovanim na fizici.

Svaki uzorak će imati 4 karakteristike:
*   **Depozit energije**: Energija ostavljena u kalorimetru (GeV)
*   **Zakrivljenost traga**: Zakrivljenost u magnetnom polju (1/GeV·c)
*   **Vreme leta**: Vreme putovanja kroz detektor (ns)
*   **Obrazac pogodaka**: Broj aktiviranih slojeva detektora

I svaki uzorak će biti označen kao:
*   **0: Elektron** (laki lepton, elektromagnetne interakcije)
*   **1: Mion** (težak lepton, minimalne interakcije)
*   **2: Pion** (mezon, jake interakcije)

Pretpostavljamo da su za svaku klasu čestica sve 4 karakteristike **normalno distribuirane**, ali sa različitim parametrima distribucije.

*Zapamtite: U pravim radnim tokovima ne generišete podatke! Prikupljate ih! To znači da ne znate prethodne distribucije. Vaš cilj je da koristite svoju ekspertizu da ih pretpostavite, a zatim izgradite najbolji model za vaše podatke.*

In [ ]:
def generate_particle_dataset(n_samples=2000):
    """
    Generiše izmišljene podatke detektora čestica sa korelacijama zasnovanim na fizici.

    Karakteristike:
    - energy_deposit: Energija ostavljena u kalorimetru (GeV)
    - track_curvature: Zakrivljenost u magnetnom polju (1/GeV·c)
    - time_of_flight: Vreme putovanja kroz detektor (ns)
    - hit_pattern: Broj aktiviranih slojeva detektora

    Klase:
    - 0: Elektron (laki lepton, elektromagnetne interakcije)
    - 1: Mion (težak lepton, minimalne interakcije)
    - 2: Pion (mezon, jake interakcije)
    """

    torch.manual_seed(42)
    np.random.seed(42)

    n_per_class = n_samples // 3

    # === ELEKTRONI ===
    # Visok depozit energije (elektromagnetni pljuskovi)
    # Visoka zakrivljenost (nizak impuls zbog radijacije)
    # Brzi (blizu brzine svetlosti)
    # Umeren obrazac pogodaka (staju u kalorimetru)

    electron_energy = torch.normal(12.0, 2.5, (n_per_class,))
    electron_curvature = torch.normal(0.85, 0.15, (n_per_class,))
    electron_tof = torch.normal(2.1, 0.2, (n_per_class,))
    electron_hits = torch.normal(8.5, 1.2, (n_per_class,))

    # === MIONI ===
    # Nizak depozit energije (minimalno jonizujuće čestice)
    # Niska zakrivljenost (visok impuls, prodirne)
    # Umerena brzina (masivna čestica)
    # Visok obrazac pogodaka (prodiru kroz ceo detektor)

    muon_energy = torch.normal(2.8, 0.8, (n_per_class,))
    muon_curvature = torch.normal(0.25, 0.08, (n_per_class,))
    muon_tof = torch.normal(2.4, 0.25, (n_per_class,))
    muon_hits = torch.normal(12.2, 1.5, (n_per_class,))

    # === PIONI ===
    # Srednji depozit energije (hadronske interakcije)
    # Varijabilna zakrivljenost (širok spektar impulsa)
    # Sporiji (teži od elektrona)
    # Varijabilan obrazac pogodaka (zavisi od interakcije)

    pion_energy = torch.normal(7.5, 3.0, (n_per_class,))
    pion_curvature = torch.normal(0

### Vizualizacija i Razumevanje Podataka

Sada kada smo simulirali prikupljanje naših podataka, možemo početi sa prvim korakom svakog ML radnog toka.

Prvi korak je uvek da **razumete podatke**!

Za ovo ćete se osloniti na dve stvari:
1. **Vaša ekspertiza u domenu** (tj. poznavanje fizike čestica, kakva ponašanja očekujete, kakve distribucije očekujete na osnovu fizike, itd.)
2. **Analiza i vizualizacija podataka** (tj. korišćenje vaših statističkih veština da analizirate važne statističke parametre podataka, i mogućnost da ih vizualizujete na najintuitivniji način)

In [ ]:
# Kreiramo vizualizaciju podataka
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
colors = ['red', 'blue', 'green']
particle_names = ['Electron', 'Muon', 'Pion']

# Distribucije karakteristika
for i, (feature_name, ax) in enumerate(zip(feature_names, axes.flat[:4])):
    for class_id, (particle_name, color) in enumerate(zip(particle_names, colors)):
        class_mask = labels == class_id
        ax.hist(features[class_mask, i].numpy(), alpha=0.6, label=particle_name,
                color=color, bins=40, density=True)
    ax.set_xlabel(feature_name)
    ax.set_ylabel('Density')
    ax.set_title(f'Distribution: {feature_name}')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Analiza korelacija
axes[1, 0].remove()
axes[1, 1].remove()
correlation_ax = fig.add_subplot(2, 3, (5, 6))

# Izračunaj matricu korelacije
correlation_matrix = torch.corrcoef(features.T)
sns.heatmap(correlation_matrix.numpy(),
            xticklabels=['Energy', 'Curvature', 'ToF', 'Hits'],
            yticklabels=['Energy', 'Curvature', 'ToF', 'Hits'],
            annot=True, cmap='coolwarm', center=0,
            ax=correlation_ax)
correlation_ax.set_title('Feature Correlations')

plt.tight_layout()
plt.show()

print("🔍 Physics Insights:")
print("📊 Electrons: High energy deposit, high curvature (low momentum)")
print("🚀 Muons: Low energy deposit, low curvature (high momentum), many hits")
print("⚛️  Pions: Intermediate properties, high variability")
print("🔗 Correlations reveal realistic physics relationships!")

---

## 7.3 Priprema podataka

Sada kada imamo i razumemo naše podatke, vreme je da pripremimo naš dataset za ML workflow.

To znači da ćemo ga spakovati kao `Dataset` potklasu, izvršiti **podelu** i **normalizaciju**, i kreirati `DataLoader` koji će za nas rukovati **batch-ovanjem** i **mešanjem**.

### Pakovanje Podataka u `Dataset` potklasu

Kao što smo naučili u Delu 2, PyTorch dolazi sa klasom koja se zove `Dataset` koja služi kao **šablon** za lakše rukovanje podacima i usmeravanje u model. Umesto direktnog rada sa sirovim podacima, kreiraćemo klasu `ParticleDataset` (potklasu od `Dataset`) koja će manipulisati našim sirovim podacima i pomoći nam sa njihovim izvlačenjem i prosleđivanjem.

In [ ]:
class ParticleDataset(Dataset):
    """
    PyTorch Dataset potklasa za klasifikaciju čestica.
    Ova klasa enkapsulira naše podatke i pruža interfejs
    koji PyTorch-ov DataLoader očekuje.
    """
    def __init__(self, features, labels, transform=None):
        """
        Args:
            features: Tenzor oblika (n_samples, n_features)
            labels: Tenzor oblika (n_samples,)
            transform: Opciona transform funkcija
        """
        self.features = features.float()  # Osiguraj float32
        self.labels = labels.long()       # Osiguraj long za klasifikaciju
        self.transform = transform

        # Čuvaj metapodatke
        self.n_samples, self.n_features = features.shape
        self.n_classes = len(torch.unique(labels))
        self.feature_names = ['energy_deposit', 'track_curvature', 'time_of_flight', 'hit_pattern']
        self.class_names = ['Electron', 'Muon', 'Pion']

    def __len__(self):
        """Vrati broj uzoraka."""
        return self.n_samples

    def __getitem__(self, idx):
        """Vrati jedan uzorak (karakteristike, label) datog indeksa."""
        sample_features = self.features[idx]
        sample_label = self.labels[idx]

        if self.transform:
            sample_features = self.transform(sample_features)

        return sample_features, sample_label

    # Sledeće su opcione metode koje će pomoći sa pripremom podataka
    def get_class_weights(self):
        """Izračunaj težine klasa za rukovanje nebalansiranim podacima."""
        class_counts = torch.bincount(self.labels)
        total_samples = len(self.labels)
        weights = total_samples / (self.n_classes * class_counts.float())
        return weights

    def get_statistics(self):
        """Vrati statistiku dataset-a."""
        return {
            'feature_means': self.features.mean(dim=0),
            'feature_stds': self.features.std(dim=0),
            'class_counts': torch.bincount(self.labels),
            'total_samples': self.n_samples
        }

# Kreiraj instancu dataset-a
full_dataset = ParticleDataset(features, labels)

print("=== PyTorch-Ready Dataset Created ===")
print(f"Dataset size: {len(full_dataset)}")
print(f"Features: {full_dataset.n_features}")
print(f"Classes: {full_dataset.n_classes}")

# Prikaži statistiku dataset-a
stats = full_dataset.get_statistics()
print(f"\nDataset Statistics:")
print(f"Feature means: {stats['feature_means']}")
print(f"Feature stds: {stats['feature_stds']}")
print(f"Class distribution: {stats['class_counts']}")

# Testiraj indeksiranje dataset-a
sample_features, sample_label = full_dataset[0]
print(f"\nSample test:")
print(f"First sample features: {sample_features}")
print(f"First sample label: {sample_label} ({full_dataset.class_names[sample_label]})")

### Podela i Normalizacija Podataka

Sada kada smo napravili čiste metode za manipulaciju našim podacima, vreme je da ih pripremimo za treniranje.

Podelićemo naše podatke na Train/Validation/Test skupove (70:15:15) i normalizovati karakteristike da osiguramo da su svi podaci na istoj skali.

In [ ]:
# Podela na train/validation/test
def create_data_splits(dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """Kreiraj train/validation/test skupove koristeći PyTorch-ov random_split."""
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6 # Odnosi moraju da se sumiraju u 1 (do na numeričku grešku na šestoj decimali)

    n_total = len(dataset)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)
    n_test = n_total - n_train - n_val  # Ovo osigurava da i n_test bude ceo broj, a da se uzorci ne ponavljaju

    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [n_train, n_val, n_test],
        generator=torch.Generator().manual_seed(42)
    )

    return train_dataset, val_dataset, test_dataset

# Kreiraj podele
train_dataset, val_dataset, test_dataset = create_data_splits(full_dataset)

print("=== Data Splitting ===")
print(f"Training samples:   {len(train_dataset):4d} ({len(train_dataset)/len(full_dataset)*100:.1f}%)")
print(f"Validation samples: {len(val_dataset):4d} ({len(val_dataset)/len(full_dataset)*100:.1f}%)")
print(f"Test samples:       {len(test_dataset):4d} ({len(test_dataset)/len(full_dataset)*100:.1f}%)")

# Normalizacija karakteristika koristeći statistiku dataset-a
def compute_normalization_stats(dataset):
    """Izračunaj statistike normalizacije iz dataset-a."""
    # Izvuci sve karakteristike iz dataset-a
    features_list = []
    for i in range(len(dataset)):
        features, _ = dataset[i]
        features_list.append(features.unsqueeze(0))

    all_features = torch.cat(features_list, dim=0)
    mean = all_features.mean(dim=0)
    std = all_features.std(dim=0)
    return mean, std

# Izračunaj normalizaciju samo iz podataka za treniranje
train_mean, train_std = compute_normalization_stats(train_dataset)

print(f"\nNormalization Statistics (from training data):")
print(f"Mean: {train_mean}")
print(f"Std:  {train_std}")

# Kreiraj transform za normalizaciju
class Normalize:
    """Transform za normalizaciju karakteristika."""
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, features):
        return (features - self.mean) / (self.std + 1e-8)  # Dodaj malo epsilon

normalize = Normalize(train_mean, train_std)

# Primeni normalizaciju na dataset-ove
class NormalizedDataset(Dataset):
    """Wrapper koji primenjuje transform normalizacije."""
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        features, label = self.dataset[idx]
        if self.transform:
            features = self.transform(features)
        return features, label

# Kreiraj normalizovane dataset-ove
train_dataset_norm = NormalizedDataset(train_dataset, normalize)
val_dataset_norm = NormalizedDataset(val_dataset, normalize)
test_dataset_norm = NormalizedDataset(test_dataset, normalize)

print("✅ Normalization applied to all splits")

### Učitavanje, Batching i Mešanje Podataka

Sada ćemo kreirati data loader-e za svaki od naša 3 podskupa dataset-a koristeći `DataLoader` klasu. `DataLoader` je napravljen da uzima `Dataset` instance i ima ugrađene metode za batch-ovanje i mešanje podataka. Takođe pruža funkcionalnost za korišćenje više CPU jezgara za paralelno učitavanje podataka (num_workers).

In [ ]:
# Kreiraj DataLoader-e
def create_dataloaders(train_dataset, val_dataset, test_dataset,
                      batch_size=64, num_workers=0):
    """Kreiraj DataLoader-e za treniranje, validaciju i testiranje."""
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,           # Pomešaj podatke za treniranje
        num_workers=num_workers,
        pin_memory=True,        # Ubrzaj GPU transfer
        drop_last=False         # Ne odbacuj nepotpune batch-ove
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,          # Nema potrebe da mešaš validaciju
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,          # Nema potrebe da mešaš test
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )

    return train_loader, val_loader, test_loader

# Kreiraj DataLoader-e
batch_size = 64
train_loader, val_loader, test_loader = create_dataloaders(
    train_dataset_norm, val_dataset_norm, test_dataset_norm,
    batch_size=batch_size
)

print("=== DataLoaders Created ===")
print(f"Batch size: {batch_size}")
print(f"Training batches:   {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches:       {len(test_loader)}")

# Testiraj DataLoader
print(f"\nDataLoader Test:")
for batch_features, batch_labels in train_loader:
    print(f"Batch features shape: {batch_features.shape}")
    print(f"Batch labels shape: {batch_labels.shape}")
    print(f"Feature range after normalization: [{batch_features.min():.3f}, {batch_features.max():.3f}]")
    print(f"Unique labels in batch: {torch.unique(batch_labels)}")
    break  # Samo prikaži prvi batch

print("🚀 Data pipeline ready for training!")

---

## 7.4 Gradnja i Treniranje Klasifikatora

Naši podaci su spremni za obradu unutar našeg modela! Međutim, prvo treba da izgradimo model.

Razmislimo o tome kakav model nam je potreban:
1. Pošto je ovo problem klasifikacije sa više opcija, pravićemo **multiklasni klasifikator** koristeći neuronsku mrežu
2. Ne želimo da naš model bude previše jednostavan i neekspresivan, ali takođe ne želimo da ga učinimo previše složenim, fleksibilnim i preekspresivnim.

Na osnovu složenosti našeg dataset-a (4 karakteristike i 3 klase), možemo početi sa nečim prilično jednostavnim, ali ipak ne previše složenim. Izaberimo mrežu sa 3 sloja (da joj damo dovoljno moći apstrakcije), i veličine 64, 32, i 16 respektivno (da obezbedimo dovoljnu rezoluciju za svaki nivo apstrakcije).

*Zapamtite: Ne postoji ispravan ili pogrešan metod za biranje ovih hiperparametara - samo pokušaji i greške pomoću informisanih pretpostavki, kao i vaša intuicija i iskustvo. Zbog toga i koristimo validaciju: da podešavamo našu mrežu na osnovu pokušaja i grešaka.*

### Građenje Arhitekture Mreže

Hajde da definišemo naš multiklasni klasifikator kao prilagođenu `nn.Module` potklasu:

In [ ]:
class ParticleClassifier(nn.Module):
    """
    Neuronska mreža za klasifikaciju čestica.
    Arhitektura dizajnirana za fizički problem:
    - Ulaz: 4 varijable iz detektora
    - Skriveni slojevi: Izdvajaju složene interakcije varijabli
    - Izlaz: 3 klase (Elektron, Mion, Pion)
    """
    def __init__(self, input_size=4, hidden_sizes=[64, 32, 16], num_classes=3, dropout_prob=0.2):
        super(ParticleClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.num_classes = num_classes
        self.dropout_prob = dropout_prob

        # Izgradi mrežu dinamički
        layers = []
        in_size = input_size

        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(in_size, hidden_size),
                nn.BatchNorm1d(hidden_size),  # Batch normalizacija za stabilnost
                nn.ReLU(),
                nn.Dropout(dropout_prob)      # Regularizacija
            ])
            in_size = hidden_size

        # Izlazni sloj (ne treba nam aktivacija na poslednjem sloju jer koristimo CrossEntropyLoss)
        layers.append(nn.Linear(in_size, num_classes))
        self.network = nn.Sequential(*layers)

        # Čuvaj metapodatke
        self.feature_names = ['energy_deposit', 'track_curvature', 'time_of_flight', 'hit_pattern']
        self.class_names = ['Electron', 'Muon', 'Pion']

        # Inicijalizuj težine pravilno
        self._initialize_weights()

    def _initialize_weights(self):
        """Inicijalizuj težine mreže koristeći najbolje prakse."""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                nn.init.constant_(module.bias, 0)

    def forward(self, x):
        """Forward pass kroz mrežu."""
        return self.network(x)

    def predict_proba(self, x):
        """Dobij verovatnoće klasa."""
        self.eval()
        with torch.no_grad():
            logits = self.forward(x)
            probabilities = F.softmax(logits, dim=1)
        return probabilities

    def predict(self, x):
        """Dobij predviđanja klasa."""
        probabilities = self.predict_proba(x)
        return torch.argmax(probabilities, dim=1)

    def get_info(self):
        """Dobij informacije o arhitekturi mreže."""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        return {
            'architecture': f"{self.input_size} → {' → '.join(map(str, self.hidden_sizes))} → {self.num_classes}",
            'total_parameters': total_params,
            'trainable_parameters': trainable_params,
            'dropout_probability': self.dropout_prob
        }

Sada kada smo dizajnirali naš `ParticleClassifier` šablon, vreme je da zapravo izgradimo (instanciramo) model:

In [ ]:
# Napravi model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ParticleClassifier(
    input_size=4,
    hidden_sizes=[64, 32, 16],
    num_classes=3,
    dropout_prob=0.2
).to(device)

print("=== Particle Classifier Created ===")
info = model.get_info()
for key, value in info.items():
    print(f"{key}: {value}")

print(f"\nModel architecture:")
print(model)

print(f"\nDevice: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

### Podešavanje Treninga

Pre nego što napravimo našu petlju za treniranje, hajde da podesimo sve komponente koje će nam biti potrebne za nju. To znači definisanje naše **funkcije gubitka**, **optimizatora**, i **scheduler-a**.

Funkcija gubitka koja najbolje odgovara multiklasnoj klasifikaciji je **Cross Entropy Loss**.

Najčešći optimizator je **Adam**, pa ćemo se držati toga.

Nismo ranije uveli scheduler-e, ali posmatrajte ih kao mehanizam koji prilagođava parametre treniranja tokom vremena da učini treniranje efikasnijim.

In [ ]:
# Podešavanje funkcije gubitka, optimizatora i scheduler-a
def create_training_components(model, train_dataset):
    """Kreiraj funkciju gubitka, optimizator i scheduler."""
    # Izračunaj težine klasa da bi se izašlo na kraj sa bilo kakvim disbalansom podataka
    class_weights = full_dataset.get_class_weights().to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # Adam optimizator sa weight decay-om (L2 regularizacija)
    optimizer = optim.Adam(
        model.parameters(),
        lr=0.001,           # Stopa učenja
        weight_decay=1e-4,  # L2 regularizacija
        betas=(0.9, 0.999)  # Adam parametri
    )

    # Scheduler stope učenja
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',         # Smanji LR kada validation loss prestane da opada
        factor=0.5,         # Pomnoži LR sa 0.5
        patience=10,        # Čekaj 10 epoha pre smanjivanja
    )

    return criterion, optimizer, scheduler

criterion, optimizer, scheduler = create_training_components(model, train_dataset)

print("=== Training Components ===")
print(f"Loss function: {criterion}")
print(f"Optimizer: {optimizer}")
print(f"Scheduler: {scheduler}")
print(f"Class weights: {criterion.weight}")

### Trening Petlja

Sada smo spremni da definišemo samu trening petlju!

Kako bi kod bio čistiji, definisaćemo šta se dešava tokom jedne epohe treniranja, a zatim definisati treniranje kao petlju po epohama:

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Treniranje jedne epohe."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        # Forward pass
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistika
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

def validate_epoch(model, val_loader, criterion, device):
    """Validiraj jednu epohu."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_features, batch_labels in val_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

Sada ćemo definisati trening sa validacijom:

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                num_epochs=100, patience=15):
    """Cela trening petlja sa ranim zaustavljanjem"""

    # Praćenje treninga
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }

    best_val_loss = float('inf')
    best_model_state = None
    epochs_without_improvement = 0

    print(f"🚀 Starting Training")
    print(f"Target: >90% validation accuracy")
    print("-" * 60)

    for epoch in range(num_epochs):
        # Faza treniranja
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)

        # Faza validacije
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)

        # Ažuriranje stope učenja
        scheduler.step(val_loss)

        # Beleži istoriju treninga
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # Logika ranog zaustavljanja
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        # Prikaži progres
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1:3d}/{num_epochs}] | "
                  f"Train: Loss={train_loss:.4f}, Acc={train_acc:.1f}% | "
                  f"Val: Loss={val_loss:.4f}, Acc={val_acc:.1f}% | "
                  f"LR={optimizer.param_groups[0]['lr']:.6f}")

        # Rano zaustavljanje
        if epochs_without_improvement >= patience:
            print(f"\n⏰ Early stopping after {epoch+1} epochs (no improvement for {patience} epochs)")
            break

    # Učitaj najbolji model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"✅ Loaded best model (val_loss={best_val_loss:.4f})")

    print("-" * 60)
    print(f"🎯 Training completed!")

    return history

Definisali smo logiku treniranja. Konačno, hajde da istreniramo naš model:

In [ ]:
# Treniraj model
training_history = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler,
    num_epochs=100, patience=15
)

### Vizualizacija Treninga

Pogledajmo kako je prošlo treniranje.

*Zapamtite: Postoje mnogo sofisticiraniji alati za praćenje i vizualizaciju treniranja koji su integrisani u samo treniranje i čine ovaj zadatak mnogo lakšim. Ali, to je van okvira ovog kursa.*

In [ ]:
# Nacrtaj istoriju treniranja
def plot_training_history(history):
    """Nacrtaj krive treniranja i validacije."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Krive gubitka
    axes[0].plot(history['train_loss'], label='Training Loss', linewidth=2)
    axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Krive tačnosti
    axes[1].plot(history['train_acc'], label='Training Accuracy', linewidth=2)
    axes[1].plot(history['val_acc'], label='Validation Accuracy', linewidth=2)
    axes[1].axhline(y=90, color='r', linestyle='--', alpha=0.7, label='Target: 90%')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Ispiši finalne rezultate
    final_train_acc = history['train_acc'][-1]
    final_val_acc = history['val_acc'][-1]
    best_val_acc = max(history['val_acc'])

    print(f"📊 Training Results:")
    print(f"   Final Training Accuracy:   {final_train_acc:.2f}%")
    print(f"   Final Validation Accuracy: {final_val_acc:.2f}%")
    print(f"   Best Validation Accuracy:  {best_val_acc:.2f}%")

    if best_val_acc >= 90:
        print(f"   🎉 SUCCESS! Target achieved!")
    else:
        print(f"   ⚠️  Target not reached (need {90-best_val_acc:.1f}% more)")

plot_training_history(training_history)

Sada, na osnovu ovih rezultata, da li želimo da promenimo nešto u našem modelu (veličinu, stopu učenja, broj slojeva, itd.) da ga učinimo boljim, ili smo zadovoljni rezultatima?

Ako niste zadovoljni, ili želite da eksperimentišete više, slobodno menjajte parametre u kodu i pokrenite treniranje ponovo.

Ako ste zadovoljni, hajde da pređemo na evaluaciju našeg modela na podacima koje nikad ranije nije video.

---

## 7.5 Evaluacija Klasifikatora

### Sveobuhvatna Evaluacija Modela

Vreme je da **testiramo naš model** davanjem podataka koje nije video tokom treniranja (posmatrajte to kao novoprikupljene podatke sa detektora, za koje je neko već izvršio ručnu analizu), i vidimo koliko dobro naš klasifikator funkcioniše:

In [ ]:
def comprehensive_evaluation(model, test_loader, device, class_names):
    """Izvršava sveobuhvatnu evaluaciju modela."""
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []

    print("🔍 Evaluating model on test set...")

    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features = batch_features.to(device)

            # Izračunaj predviđanja i verovatnoće
            outputs = model(batch_features)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(batch_labels.numpy())
            all_probabilities.extend(probabilities.cpu().numpy())

    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    all_probabilities = np.array(all_probabilities)

    # Izračunaj metrike
    test_accuracy = 100 * (all_predictions == all_labels).mean()
    print(f"🎯 Test Accuracy: {test_accuracy:.2f}%")
    print("-" * 50)

    # Detaljan izveštaj klasifikacije
    print("📋 Classification Report:")
    print(classification_report(all_labels, all_predictions, target_names=class_names))

    # Matrica konfuzije
    cm = confusion_matrix(all_labels, all_predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Analiza po klasama
    print("🔬 Per-Class Analysis:")
    for i, class_name in enumerate(class_names):
        class_mask = all_labels == i
        class_accuracy = (all_predictions[class_mask] == i).mean() * 100
        class_confidence = all_probabilities[class_mask, i].mean()
        print(f"   {class_name:8s}: Accuracy={class_accuracy:.1f}%, Avg Confidence={class_confidence:.3f}")

    return {
        'test_accuracy': test_accuracy,
        'predictions': all_predictions,
        'labels': all_labels,
        'probabilities': all_probabilities,
        'confusion_matrix': cm
    }

# Evaluiraj model
evaluation_results = comprehensive_evaluation(model, test_loader, device, model.class_names)

### Analiza sa Stanovišta Fizike

U redu, sada imamo rezultate i njihovu statističku analizu, ali kako da interpretiramo ovo iz perspektive našeg domena (fizike)?

Uvek je važno razumeti rezultate i performanse, bilo da su dobre ili loše.

In [ ]:
def physics_analysis(model, test_loader, device, feature_names, class_names):
    """Analiziraj predviđanja modela iz fizičke perspektive."""
    model.eval()
    feature_data = []
    predictions_data = []
    labels_data = []
    confidence_data = []

    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features_norm = batch_features.to(device)

            # Izračunaj predviđanja
            outputs = model(batch_features_norm)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            confidence = torch.max(probabilities, 1)[0]

            # Čuvaj podatke (denormalizuj karakteristike za interpretaciju)
            batch_features_orig = batch_features * train_std + train_mean
            feature_data.extend(batch_features_orig.numpy())
            predictions_data.extend(predicted.cpu().numpy())
            labels_data.extend(batch_labels.numpy())
            confidence_data.extend(confidence.cpu().numpy())

    feature_data = np.array(feature_data)
    predictions_data = np.array(predictions_data)
    labels_data = np.array(labels_data)
    confidence_data = np.array(confidence_data)

    print("🔬 Physics-Informed Analysis")
    print("-" * 40)

    # Analiziraj pogrešne klasifikacije
    misclassified = predictions_data != labels_data
    print(f"Misclassified samples: {misclassified.sum()} / {len(labels_data)} ({misclassified.mean()*100:.1f}%)")

    if misclassified.sum() > 0:
        print("\nMisclassification Analysis:")
        for true_class in range(3):
            for pred_class in range(3):
                if true_class != pred_class:
                    mask = (labels_data == true_class) & (predictions_data == pred_class)
                    count = mask.sum()
                    if count > 0:
                        avg_confidence = confidence_data[mask].mean()
                        print(f"   {class_names[true_class]} → {class_names[pred_class]}: "
                              f"{count:3d} samples (confidence: {avg_confidence:.3f})")

    # Analiza važnosti karakteristika
    print(f"\n📊 Feature Analysis by Particle Type:")
    for class_id, class_name in enumerate(class_names):
        class_mask = labels_data == class_id
        print(f"\n{class_name} characteristics:")
        for feat_id, feat_name in enumerate(feature_names):
            mean_val = feature_data[class_mask, feat_id].mean()
            std_val = feature_data[class_mask, feat_id].std()
            print(f"   {feat_name:20s}: {mean_val:6.2f} ± {std_val:5.2f}")

    # Analiza pouzdanosti
    print(f"\n🎯 Model Confidence Analysis:")
    for class_id, class_name in enumerate(class_names):
        class_mask = predictions_data == class_id
        if class_mask.sum() > 0:
            avg_confidence = confidence_data[class_mask].mean()
            print(f"   {class_name:8s} predictions: {avg_confidence:.3f} average confidence")

# Izvrši analizu sa stanovišta fizike
physics_analysis(model, test_loader, device, full_dataset.feature_names, model.class_names)

Šta možete da kažete o našem modelu i fizici koju smo naučili iz njega?

---

## 7.6 Izvoz i Implementacija Modela

Kada ste istrenirali i evaluirali svoj model i osećate da je spreman za korišćenje kao alat, treba da ga **implementirate**. Postoji nekoliko biblioteka koje rukuju čuvanjem, izvozom i implementacijom modela, kao i učitavanjem i korišćenjem u drugim projektima. Međutim, koristićemo PyTorch-ove ugrađene metode za ovo.

### Čuvanje Modela

In [ ]:
# Sačuvaj istrenirani model
def save_model(model, filepath, training_history, normalization_stats):
    """Save model with all necessary information for deployment."""

    save_dict = {
        'model_state_dict': model.state_dict(),
        'model_config': model.get_info(),
        'training_history': training_history,
        'normalization_mean': normalization_stats[0],
        'normalization_std': normalization_stats[1],
        'class_names': model.class_names,
        'feature_names': model.feature_names
    }

    torch.save(save_dict, filepath)
    print(f"💾 Model saved to: {filepath}")

# Sačuvaj model
model_save_path = "particle_classifier.pth"
save_model(model, model_save_path, training_history, (train_mean, train_std))

### Učitavanje i Korišćenje Modela

In [ ]:
def load_and_predict(filepath, sample_features):
    """Učitaj model i napravi predviđanja na novim podacima."""
    # Učitaj model
    save_dict = torch.load(filepath, map_location=device)

    # Rekreiraj arhitekturu modela na osnovu sačuvanih podataka
    loaded_model = ParticleClassifier().to(device)
    loaded_model.load_state_dict(save_dict['model_state_dict'])
    loaded_model.eval()

    # Izračunaj statistiku normalizacije
    norm_mean = save_dict['normalization_mean']
    norm_std = save_dict['normalization_std']
    class_names = save_dict['class_names']

    # Normalizuj ulaz
    sample_normalized = (sample_features - norm_mean) / norm_std
    sample_normalized = sample_normalized.to(device)

    # Napravi predviđanje
    with torch.no_grad():
        outputs = loaded_model(sample_normalized.unsqueeze(0))
        probabilities = F.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities.max().item()

    return {
        'predicted_class': predicted_class,
        'predicted_particle': class_names[predicted_class],
        'confidence': confidence,
        'all_probabilities': probabilities.squeeze().cpu().numpy()
    }

### Testiranje Implementacije

In [ ]:
# Testiraj Implementaciju
print("\n🚀 Testing Model Deployment:")
print("-" * 30)

# Napravi nekoliko uzoraka očitavanja iz detektora
sample_readings = torch.tensor([
    [10.5, 0.75, 2.2, 8.0],  # Liči na elektron
    [2.5, 0.20, 2.4, 12.0], # Liči na mion
    [6.8, 0.50, 2.9, 7.0]   # Liči na pion
]).to(device)

for i, reading in enumerate(sample_readings):
    result = load_and_predict(model_save_path, reading)
    print(f"Sample {i+1}: {result['predicted_particle']} "
          f"(confidence: {result['confidence']:.3f})")
    print(f"   Probabilities: Electron={result['all_probabilities'][0]:.3f}, "
          f"Muon={result['all_probabilities'][1]:.3f}, "
          f"Pion={result['all_probabilities'][2]:.3f}")

print("\n✅ Model successfully deployed and tested!")

---

# Ključni Zaključci

## Pristup rešavanju problema:

1. Razumeti problem
2. Prikupiti i razumeti podatke
3. Koristiti profesionalne PyTorch radne tokove za podatke
4. Dizajnirati odgovarajuću arhitekturu mreže na osnovu poznavanja problema i podataka
5. Trenirati koristeći najbolje navike (normalizacija, regularizacija)
6. Sveobuhvatno evaluirati
7. Implementirati sa pravilnim čuvanjem/učitavanjem modela

## Najbolje PyTorch navike:

- Koristite Dataset i DataLoader klase
- Podelite podatke na trening/validaciju/test skupove
- Normalizacijujte karakteristike
- Odaberite dobru veličinu batch-a
- Koristite GPU-ove kad god možete
- Primenite rano zaustavljanje i scheduling stope učenja
- Koristite sveobuhvatne metrike za evaluaciju
- Pravilno izvezite modele za dalju implementaciju

## Uvid iz Fizike:

- Neuronske mreže mogu da nauče složene procese bez analitičkih rešenja
- Korelacije karakteristika odražavaju pravu fiziku
- Interpretabilnost modela je ključna u nauci
- Analiza pouzdanosti pomaže identifikaciji sistematskih grešaka

## Sledeći koraci:

Na sledećem kursu (KSMF2), naučićemo više o složenijim arhitekturama modela koje su osnova za moderni AI. Naučićemo kako da ih implementiramo koristeći složenije PyTorch apstrakcije i još profesionalnije pipeline-ove sa više GPU-ova, Torch Lightning, itd.

---

# Samostalni Rad

Kao vežbu, pokušajte da primenite sve što ste naučili u pravljenju vaših sopstvenih modela za stvarne, ili izmišljene primere. Možete čak proći kroz ovaj primer i pokušati da eksperimentišete sa dataset-om, modelom, treniranjem i evaluacijom.